In [433]:
import numpy as np
from scipy.optimize import newton
import pandas as pd

In [434]:
typical_unit = pd.read_csv('typical_unit.csv')
typical_unit

,INDEX,CR,FAR,BA,BN,UA,RA,AVGH,MAXH,OFF,COM,IND,PUB,RES,ASHP_N,battery_N,HP_price
0,4271,0.027897,0.111264,136691.32120,183,1.228500e+06,34272.72676,12.000000,12,100.000000,0.000000,0.000000,0.000000,0.000000,159,39,6.112000e+06
1,13775,0.216268,4.273060,339996.26320,30,7.956741e+04,17207.90835,44.142857,108,88.317062,0.000000,0.000000,0.000000,11.682938,173,20,1.051333e+07
2,13567,0.098981,0.370199,58531.90510,29,1.581092e+05,15649.79495,11.357143,18,53.725940,0.000000,0.000000,0.000000,46.274060,44,18,2.121781e+06
3,16018,0.150972,0.668662,123855.72710,85,1.852291e+05,27964.35319,12.000000,24,0.000000,0.000000,0.000000,0.000000,100.000000,110,62,5.843454e+06
4,19644,0.225034,2.916986,266584.25390,28,9.139030e+04,20565.93084,32.888889,90,0.000000,0.000000,0.000000,0.000000,100.000000,167,44,6.405214e+06
5,3382,0.199157,5.331481,473699.50270,27,8.884952e+04,17695.04104,71.076923,126,0.000000,0.000000,0.000000,0.000000,100.000000,264,16,1.085605e+07
6,10712,0.124118,0.500644,80253.14086,20,1.602998e+05,19896.14940,12.315789,18,0.000000,0.000000,100.000000,0.000000,0.000000,11,23,1.810556e+06
7,19515,0.151963,2.414846,264906.10670,24,1.096990e+05,16670.19812,43.043478,90,0.000000,0.000000,100.000000,0.000000,0.000000,110,17,6.009773e+06
8,19565,0.087295,0.318596,418829.09410,391,1.314600e+06,114758.47110,10.431818,18,0.000000,0.000000,100.000000,0.000000,0.000000,136,127,1.126634e+07
9,19558,0.054617,0.195879,53787.02350,44,2.745937e+05,14997.44775,11.121951,18,0.558196,0.000000,0.000000,99.441804,0.000000,34,17,2.081673e+06


In [435]:
site_id = 16046

testData = pd.read_csv('./peking_unit/'+str(site_id)+'_posEInPV.csv')

testData

,consumption,pv,price_buy,price_sell,posEInPV
0,987.575409,0.0,0.130182,0.3598,0.0
1,994.811815,0.0,0.130182,0.3598,0.0
2,869.200553,0.0,0.130182,0.3598,0.0
3,1091.994657,0.0,0.130182,0.3598,0.0
4,1171.465794,0.0,0.130182,0.3598,0.0
...,...,...,...,...,...
8755,1964.597108,0.0,0.841935,0.3598,0.0
8756,1877.382327,0.0,0.841935,0.3598,0.0
8757,1562.284899,0.0,0.701612,0.3598,0.0
8758,919.560435,0.0,0.701612,0.3598,0.0


In [436]:
import os
import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# 指定文件夹路径
folder_path = r'D:\iFHHs\课题组\毕设\finalterm\energy_data'

# 生成待读取文件夹路径
folder_name = str(site_id) + '_ASHP'
folder_path = os.path.join(folder_path, folder_name)

# 列出文件夹中所有xlsx文件
xlsx_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx') and re.match(r'^\d', file)]

# 创建空的DataFrame用于存储提取的数据
result_df = pd.DataFrame()

# 使用 tqdm 来跟踪文件读取进度
for file in tqdm(xlsx_files, desc='Reading files'):
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path, usecols=[1, 4, 5, 6], header=None)  # 仅提取第二列数据
    result_df = pd.concat([result_df, df], axis=1)  # 将提取的数据列拼接到结果DataFrame中

result_df

Reading files: 100%|██████████| 65/65 [00:23<00:00,  2.71it/s]


,1,4,5,6,1,4,5,6,1,4,...,5,6,1,4,5,6,1,4,5,6
0,11.525352,0.000000,0.322387,0.091200,9.432356,0.027055,0.45481,0.069313,14.045147,0.046189,...,0.466563,0.060743,9.068986,0.028961,0.476204,0.068571,7.340261,0.021654,0.362322,0.052900
1,11.190860,0.000000,0.322387,0.090516,9.737769,0.034097,0.45481,0.071471,14.033486,0.058142,...,0.466563,0.062359,9.172294,0.036634,0.476204,0.068602,7.477886,0.027321,0.362322,0.053499
2,9.465824,0.000000,0.322387,0.069356,8.064421,0.034097,0.45481,0.053495,11.498146,0.058142,...,0.466563,0.048383,7.576730,0.036634,0.476204,0.051139,6.182678,0.027541,0.362322,0.040750
3,11.918817,0.000000,0.322387,0.095607,11.754159,0.027277,0.45481,0.082876,16.081611,0.046513,...,0.466563,0.071887,10.607985,0.029307,0.476204,0.076524,8.514704,0.022033,0.362322,0.060203
4,12.655774,0.000000,0.322387,0.105761,13.044902,0.027277,0.45481,0.097672,17.371104,0.046513,...,0.466563,0.087083,11.460579,0.029307,0.476204,0.089071,9.516406,0.022033,0.362322,0.071960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,13.068588,0.568344,0.322387,0.116019,15.458477,0.231425,0.45481,0.119392,21.478768,0.394628,...,0.466563,0.081747,12.939174,0.248648,0.476204,0.101774,10.554321,0.186932,0.362322,0.080302
8756,13.204055,0.551628,0.322387,0.118055,15.969680,0.224619,0.45481,0.125170,22.475792,0.383021,...,0.466563,0.090557,13.823748,0.241335,0.476204,0.105196,11.299977,0.181434,0.362322,0.085114
8757,7.254674,0.000000,0.322387,0.067807,20.435141,0.156552,0.45481,0.173326,29.887636,0.266954,...,0.466563,0.147687,18.961740,0.168203,0.476204,0.166229,15.235118,0.126454,0.362322,0.128462
8758,8.904940,0.000000,0.322387,0.078556,8.155055,0.088486,0.45481,0.069418,13.356458,0.150887,...,0.466563,0.062389,8.459693,0.095071,0.476204,0.073859,6.469423,0.071474,0.362322,0.054467


#### 热泵的价格变化

In [437]:
HP_E = result_df.sum(axis=1)

sellPrice = testData['price_sell'].values

delta_HP_price = - (sellPrice * HP_E).sum() + typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'HP_price']

# delta_HP_price 
print(int(typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'HP_price']))
print(int((sellPrice * HP_E).sum()))
print('HP_save:',delta_HP_price)
print(100 - round((sellPrice * HP_E).sum()/typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'HP_price']*100,1),'%')


3153779
1004284
HP_save: 2149495.889330717
68.2 %


#### 逐时数据准备

In [438]:
# 判断光伏自用、电网供电和光伏上网

def classification(site_id):
    site_id['PVcharge'] = site_id.apply(lambda x: min(x['consumption'], x['pv']), axis=1)
    site_id['gridcharge'] = site_id['consumption'] - site_id['PVcharge']
    
    site_id['PV2G'] = site_id['pv']-site_id['PVcharge']-site_id['posEInPV']
    return site_id

classified_new = classification(testData)

# classified_SC0.to_csv('classified_SC0.csv', index=False)

In [439]:
classified_new

,consumption,pv,price_buy,price_sell,posEInPV,PVcharge,gridcharge,PV2G
0,987.575409,0.0,0.130182,0.3598,0.0,0.0,987.575409,0.0
1,994.811815,0.0,0.130182,0.3598,0.0,0.0,994.811815,0.0
2,869.200553,0.0,0.130182,0.3598,0.0,0.0,869.200553,0.0
3,1091.994657,0.0,0.130182,0.3598,0.0,0.0,1091.994657,0.0
4,1171.465794,0.0,0.130182,0.3598,0.0,0.0,1171.465794,0.0
...,...,...,...,...,...,...,...,...
8755,1964.597108,0.0,0.841935,0.3598,0.0,0.0,1964.597108,0.0
8756,1877.382327,0.0,0.841935,0.3598,0.0,0.0,1877.382327,0.0
8757,1562.284899,0.0,0.701612,0.3598,0.0,0.0,1562.284899,0.0
8758,919.560435,0.0,0.701612,0.3598,0.0,0.0,919.560435,0.0


In [440]:
testData['posEInPV'].sum()

27656.719254453772

#### 能量

In [441]:
# 求自消耗率

pvcharge_sum = testData['PVcharge'].sum()
pv_sum = testData['pv'].sum()
posEInPV = testData['posEInPV'].sum()

# 计算 PVcharge 列和 PV 列的比值
Self_consumption = (pvcharge_sum + posEInPV)/ pv_sum
print("Self_consumption:", round(Self_consumption,2))

Self_consumption: 0.95


光伏自消耗率：指光伏发电系统中，自身消耗的电力占总发电量的比例。光伏自消耗率越低，说明光伏发电系统的能源利用率越高，系统运行效率越好。

In [442]:
# 求自满足率

pvcharge_sum = testData['PVcharge'].sum()
en_sum = testData['consumption'].sum()
posEInPV = testData['posEInPV'].sum()

# 计算 PVcharge 列和 PV 列的比值
Self_consumption = (pvcharge_sum + posEInPV) / en_sum
print("Self_sufficiency:", round(Self_consumption,2))

Self_sufficiency: 0.35


光伏自满足率，指光伏发电系统中，自身能够满足的电力需求占总需求量的比例。光伏自满足率越高，说明光伏发电系统对自身电力需求的满足程度越好，从而减少了对传统能源的依赖，提高了经济性和环保性。

#### 环境

In [443]:
# 运行阶段碳排放
# 假设已经获得了光伏发电和能耗数据

total_energy_consumption = testData['consumption'].sum()

emission_factor_electricity = 0.615 # kgCO2/kWh

# 计算全年运行阶段碳排放量
total_emission = total_energy_consumption * emission_factor_electricity


print("Total energy consumption:", total_energy_consumption)
print("Total carbon emission:", total_emission)
print("Average CO2:", round((total_energy_consumption * emission_factor_electricity)/typical_unit[typical_unit['INDEX'] == site_id]['BA'].values[0],2))

Total energy consumption: 12062648.867666999
Total carbon emission: 7418529.053615204
Average CO2: 82.99


#### 经济

In [444]:
# 自发自用省的,光伏上网赚的,电池节省的，三部分数据已经在Optimizing_peking中计算，这里直接取变量

%store -r benifit cost_without_system cost_without_batt cost_with_batt
inflows = (benifit + delta_HP_price )/ 10000 # 用光伏后每年的收益，单位：万元


print('Cost without pv:',round(cost_without_system/typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'BA'],2))
print ('Cost without battery:', round(cost_without_batt/typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'BA'],2))
print ('Cost with battery:', round(cost_with_batt/typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'BA'],2),'\n')


print ('Score_pv: %.1f'%((cost_without_system - cost_without_batt)*100 / np.abs(cost_without_system)),'%')
print ('Score_battery: %.1f'%((cost_without_system - cost_with_batt)*100 / np.abs(cost_without_system)),'%')

Cost without pv: 69.44
Cost without battery: 41.66
Cost with battery: 40.82 

Score_pv: 40.0 %
Score_battery: 41.2 %


#### 此处调用面积的时候需要注意对应顺序

#### 计算IRR

In [445]:
import numpy_financial as npf
from scipy.optimize import fsolve

# 初始投资
initial_investment = -(typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'RA'] * 1600
                        + 80000 * typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'ASHP_N'] 
                        + 53000 * typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'battery_N'] )/10000  # 负数表示投资支出，单位万元

# 每年净收入，包括初始投资后的收入
net_inflows = [inflows] * 25  # 创建包含25个年收入的列表

# 计算IRR函数
def irr_func(rate):
    return npf.npv(rate, [initial_investment] + net_inflows)

# 使用fsolve函数计算IRR
irr = fsolve(irr_func, 0.1)[0]  # 初始猜测IRR为0.1，可根据实际情况调整

# 计算payback year
cumulative_cashflows = [initial_investment]  # 初始投资先作为第一个累积现金流
for i in range(1, len(net_inflows)):
    cumulative_cashflow = cumulative_cashflows[i - 1] + net_inflows[i]
    cumulative_cashflows.append(cumulative_cashflow)
    if cumulative_cashflow >= 0:
        payback_year = i + 1  # 加1是因为列表索引从0开始，而年份从1开始
        break

print("计算得到的IRR:", round(irr*100,1),'%')
print("计算得到的Payback Year:", payback_year)

计算得到的IRR: 9.2 %
计算得到的Payback Year: 11


IRR分布在8%左右，能得到不错的对比

#### 计算LCOE

In [446]:
typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'RA'] *3000/ 50

1340804.8895999999

In [447]:
def calculate_lcoe(capital_cost, initial_opex, life, annual_energy_production, inflation_rate):

    total_cost_present_value = capital_cost
    opex_sum = 0
    annual_energy_sum = 0

    for year in range(1, life + 1):
        if year == 10 or year == 20: # 第十年和二十年逆变器更换
            total_cost_present_value += (initial_opex + (typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'RA'] *6900/ 50)) / (1 + inflation_rate) ** year
            opex_sum += (initial_opex + (typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'RA'] *6900/ 50)) / (1 + inflation_rate) ** year
        else:
            total_cost_present_value += initial_opex / (1 + inflation_rate) ** year
            opex_sum += initial_opex / (1 + inflation_rate) ** year
        annual_energy_sum += annual_energy_production / (1 + inflation_rate) ** year

    lcoe = total_cost_present_value / annual_energy_sum
    return lcoe

capital_cost = typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'RA'] * 1600 # 每平米价格大约1000元
initial_opex = typical_unit.loc[typical_unit.loc[typical_unit['INDEX'] == site_id].index[0], 'RA'] * 16 # 清洗和更换其余部件的费用为初始投资额的1%
life = 25
annual_energy_production = testData['pv'].sum()
inflation_rate = 0.025

lcoe = calculate_lcoe(capital_cost, initial_opex, life, annual_energy_production, inflation_rate)
print("LCOE:", round(lcoe, 4), "CNY/kWh")


LCOE: 0.5787 CNY/kWh


分布式光伏度电成本低于当地工商业用户的电价。说明用户方在目前的技术和成本水平下已经能够接入平价。从发电端来看，分布式光伏度电成本均高于当地脱硫煤价。揭示出，在目前的技术水平下，分布式光伏在发电侧实现平价上网仍存在一定难度。

并且遮挡率和LCOE成正相关关系，表示了在同一个地区下，社区形态的复杂性带来的不同遮挡率对LCOE存在明显的影响，需要避开遮挡率大的社区。

#### 韧性

In [448]:
RE = (testData['pv'].sum()) / (testData['consumption'].sum())
print('RE = ',round(RE * 100, 2),'%')

RE =  36.26 %
